GitHub<br>
https://github.com/Time-Travel-Rephotography/Time-Travel-Rephotography.github.io<br>
論文<br>
https://arxiv.org/abs/2012.12261<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/Time_Travel_Rephotography_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## Githubからソースコード取得

In [ ]:
%cd /content

!git clone  --depth 1 --recurse-submodules --shallow-submodules \
  https://github.com/Time-Travel-Rephotography/Time-Travel-Rephotography.github.io.git Time-Travel-Rephotography

# for face align
!git clone https://github.com/adamian98/pulse.git

## ライブラリのインストール

In [ ]:
%cd /content/Time-Travel-Rephotography

!pip3 install -r requirements.txt
!pip3 install --upgrade gdown

## ライブラリのインポート

In [ ]:
%cd /content/Time-Travel-Rephotography

from pathlib import Path
import os

from PIL import Image
from IPython.display import display

from argparse import Namespace
from projector import (
    ProjectorArguments,
    main,
)

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/Time-Travel-Rephotography
!mkdir -p ./checkpoint/encoder

if not os.path.exists('checkpoint/e4e_ffhq_encode.pt'):
  !gdown https://drive.google.com/uc?id=1YLB-3ZCv6FRAWwCHkTvUou_CAvDcb5Pr -O checkpoint/e4e_ffhq_encode.pt
if not os.path.exists('checkpoint/stylegan2-ffhq-config-f.pt'):
  !gdown https://drive.google.com/uc?id=1aSnTVHGNzQ-Eh5rNDOrOc_4SjQbNMq3B -O checkpoint/stylegan2-ffhq-config-f.pt
if not os.path.exists('checkpoint/vgg_face_dag.pt'):
  !gdown https://drive.google.com/uc?id=12BHlsSQM0D8KyXprIc7WmJRLWBWdtF56 -O checkpoint/vgg_face_dag.pt

if not os.path.exists('checkpoint/checkpoint_b.pt'):
  !gdown https://drive.google.com/uc?id=1fXBCyBKNEZfeiI6LEHTEnUg_DjWHJy5r -O checkpoint/encoder/checkpoint_b.pt
if not os.path.exists('checkpoint/checkpoint_g.pt'):
  !gdown https://drive.google.com/uc?id=1YnQEPf7FyfZxAVQg-CpbvdF4otZeTOZh -O checkpoint/encoder/checkpoint_g.pt
if not os.path.exists('checkpoint/checkpoint_gb.pt'):
  !gdown https://drive.google.com/uc?id=1kelQK3pUdeHwu8uVE7A4WZH_EjHxGE6v -O checkpoint/encoder/checkpoint_gb.pt

%cd /content/Time-Travel-Rephotography/third_party/face_parsing
!mkdir -p res/cp

if not os.path.exists('checkpoint/res/cp/79999_iter.pth'):
  !gdown https://drive.google.com/uc?id=1vwm4BcAKISQgcJLvTUcvesk73UIYdDMF -O res/cp/79999_iter.pth

# テスト画像のセットアップ

In [ ]:
%cd /content/Time-Travel-Rephotography
!mkdir test_imgs

# default
# input_path = "/content/Time-Travel-Rephotography/dataset/Abraham Lincoln_01.png"
# アインシュタイン
# !wget https://astro-dic.jp/wp/wp-content/uploads/A_Einstein-1.jpg -O ./test_imgs/test.jpg
# マリリンモンロー
!wget https://www.allcinema.net/img/6/6476/p_40178_01_01_02.jpg -O ./test_imgs/test.jpg

## 顔部分切り取り

In [ ]:
%cd /content/pulse/

!python align_face.py \
  -input_dir /content/Time-Travel-Rephotography/test_imgs \
  -output_dir  /content/Time-Travel-Rephotography/test_aligns \
  -output_size 512 \
  -seed 12 \

In [ ]:
%cd /content/Time-Travel-Rephotography

input_path = '/content/Time-Travel-Rephotography/test_aligns/test_0.png'

display(Image.open(input_path))

# Prompts

In [ ]:
#@markdown 'b' (blue-sensitive), 'gb' (orthochromatic), 'g' (panchromatic)
spectral_sensitivity = "b" # @param ["b", "gb", "g"]
#@markdown Estimated blur radius of the input photo
gaussian_radius =   0.75 # @param {type:"number"}

In [ ]:
%cd /content/Time-Travel-Rephotography

args = ProjectorArguments().parse(
    args=[str(input_path)], 
    namespace=Namespace(
        spectral_sensitivity=spectral_sensitivity,
        encoder_ckpt=f"checkpoint/encoder/checkpoint_{spectral_sensitivity}.pt",
        encoder_name=spectral_sensitivity,
        gaussian=gaussian_radius,
        log_visual_freq=1000,
        log_dir="log/",
        results_dir="results/"
))
main(args)

In [ ]:
def get_concat_h_multi_resize(im_list, resample=Image.BICUBIC):
    min_height = min(im.height for im in im_list)
    im_list_resize = [im.resize((int(im.width * min_height / im.height), min_height),resample=resample)
                      for im in im_list]
    total_width = sum(im.width for im in im_list_resize)
    dst = Image.new('RGB', (total_width, min_height))
    pos_x = 0
    for im in im_list_resize:
        dst.paste(im, (pos_x, 0))
        pos_x += im.width
    return dst

im1 = Image.open(f"/content/Time-Travel-Rephotography/results/test_0_{spectral_sensitivity}.png")
im2 = Image.open('/content/Time-Travel-Rephotography/results/test_0/skin_mask/face_parsing/input.jpg')
im3 = Image.open(f"/content/Time-Travel-Rephotography/results/test_0-{spectral_sensitivity}-G{gaussian_radius}-init(10,18)-s256-vgg1-vggface0.3-eye0.1-color1.0e+10-cx0.1(relu3_4,relu2_2,relu1_2)-NR5.0e+04-lr0.1_0.01-c32-wp(250,750).png")

concat_img = get_concat_h_multi_resize([im1, im2, im3])

display(concat_img)